# ശ്രദ്ധാ യന്ത്രങ്ങളും ട്രാൻസ്ഫോർമറുകളും

പുനരാവർത്തന ശൃംഖലകളുടെ ഒരു പ്രധാന ദോഷം, ഒരു ശൃംഖലയിൽ ഉള്ള എല്ലാ വാക്കുകളും ഫലത്തിൽ ഒരുപോലെ സ്വാധീനം ചെലുത്തുന്നതാണ്. ഇത് നാമനിർദ്ദേശം തിരിച്ചറിയൽ, യന്ത്രഭാഷാന്തരം പോലുള്ള ശൃംഖല-തെ shൃംഖല പ്രവർത്തനങ്ങളിൽ സാധാരണ LSTM എൻകോഡർ-ഡികോഡർ മോഡലുകളുടെ പ്രകടനം കുറയ്ക്കുന്നു. യാഥാർത്ഥ്യത്തിൽ, ഇൻപുട്ട് ശൃംഖലയിൽ ചില പ്രത്യേക വാക്കുകൾ പരമ്പരാഗത ഔട്ട്പുട്ടുകളിൽ മറ്റുള്ളവയെക്കാൾ കൂടുതൽ സ്വാധീനം ചെലുത്താറുണ്ട്.

യന്ത്രഭാഷാന്തരം പോലുള്ള ശൃംഖല-തെ shൃംഖല മോഡൽ പരിഗണിക്കാം. ഇത് രണ്ട് പുനരാവർത്തന ശൃംഖലകളാൽ നടപ്പിലാക്കപ്പെടുന്നു, ഒരു ശൃംഖല (**എൻകോഡർ**) ഇൻപുട്ട് ശൃംഖലയെ ഹിഡൻ സ്റ്റേറ്റിലേക്ക് ചുരുക്കും, മറ്റൊന്ന്, **ഡികോഡർ**, ഈ ഹിഡൻ സ്റ്റേറ്റ് വിവർത്തനം ചെയ്ത ഫലമായി തുറക്കും. ഈ സമീപനത്തിലെ പ്രശ്നം, ശൃംഖലയുടെ അന്തിമ സ്റ്റേറ്റ് ഒരു വാക്യത്തിന്റെ തുടക്കം ഓർക്കാൻ ബുദ്ധിമുട്ട് അനുഭവപ്പെടുന്നതാണ്, അതിനാൽ ദീർഘ വാക്യങ്ങളിൽ മോഡലിന്റെ ഗുണമേന്മ കുറയുന്നു.

**ശ്രദ്ധാ യന്ത്രങ്ങൾ** RNN-ന്റെ ഓരോ ഔട്ട്പുട്ട് പ്രവചനത്തിലും ഓരോ ഇൻപുട്ട് വെക്ടറിന്റെ സാന്ദർഭിക സ്വാധീനം തൂക്കമിടാനുള്ള മാർഗം നൽകുന്നു. ഇത് നടപ്പിലാക്കുന്നത് ഇൻപുട്ട് RNN-ന്റെ ഇടനില സ്റ്റേറ്റുകളും ഔട്ട്പുട്ട് RNN-ന്റെ ഇടനില സ്റ്റേറ്റുകളും തമ്മിൽ ഷോർട്ട്കട്ടുകൾ സൃഷ്ടിച്ച് ആണ്. ഈ രീതിയിൽ, ഔട്ട്പുട്ട് ചിഹ്നം $y_t$ സൃഷ്ടിക്കുമ്പോൾ, വ്യത്യസ്ത തൂക്കം കോഫിഷ്യന്റുകളായ $\alpha_{t,i}$ ഉപയോഗിച്ച് എല്ലാ ഇൻപുട്ട് ഹിഡൻ സ്റ്റേറ്റുകളും $h_i$ പരിഗണിക്കും.

![Image showing an encoder/decoder model with an additive attention layer](../../../../../translated_images/ml/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) ലെ കൂട്ടിച്ചേർത്ത ശ്രദ്ധാ യന്ത്രം ഉള്ള എൻകോഡർ-ഡികോഡർ മോഡൽ, [ഈ ബ്ലോഗ് പോസ്റ്റ്](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) നിന്നെടുത്തത്]*

ശ്രദ്ധാ മാട്രിക്സ് $\{\alpha_{i,j}\}$ ഒരു പ്രത്യേക ഇൻപുട്ട് വാക്ക് ഔട്ട്പുട്ട് ശൃംഖലയിൽ ഒരു വാക്ക് സൃഷ്ടിക്കുന്നതിൽ എത്രമാത്രം പങ്കുവഹിക്കുന്നുവെന്ന് പ്രതിനിധീകരിക്കും. താഴെ ഒരു ഉദാഹരണ മാട്രിക്സ് കാണിക്കുന്നു:

![Image showing a sample alignment found by RNNsearch-50, taken from Bahdanau - arviz.org](../../../../../translated_images/ml/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (ചിത്രം 3) നിന്നെടുത്ത ചിത്രം]*

ശ്രദ്ധാ യന്ത്രങ്ങൾ പ്രകൃതിഭാഷാ പ്രോസസ്സിംഗിലെ ഇപ്പോഴത്തെ അല്ലെങ്കിൽ അടുത്തകാലത്തെ ഏറ്റവും മികച്ച നിലവാരത്തിന് വലിയ പങ്ക് വഹിക്കുന്നു. എന്നാൽ, ശ്രദ്ധ ചേർക്കുന്നത് മോഡൽ പാരാമീറ്ററുകളുടെ എണ്ണം വളരെ വർദ്ധിപ്പിക്കുന്നു, ഇത് RNN-കളുടെ സ്കെയിലിംഗ് പ്രശ്നങ്ങൾക്ക് കാരണമാകുന്നു. RNN-കളുടെ സ്കെയിലിംഗിന്റെ പ്രധാന തടസ്സം, മോഡലുകളുടെ പുനരാവർത്തന സ്വഭാവം കാരണം പരിശീലനം ബാച്ച് ചെയ്യാനും പാരലലൈസ് ചെയ്യാനും ബുദ്ധിമുട്ട് ഉണ്ടാകുന്നതാണ്. RNN-ൽ ഒരു ശൃംഖലയുടെ ഓരോ ഘടകവും പരമ്പരാഗത ക്രമത്തിൽ പ്രോസസ്സ് ചെയ്യേണ്ടതുണ്ട്, അതിനാൽ ഇത് എളുപ്പത്തിൽ പാരലലൈസ് ചെയ്യാനാകില്ല.

ശ്രദ്ധാ യന്ത്രങ്ങളുടെ സ്വീകരണവും ഈ തടസ്സവും ചേർന്ന്, നാം ഇന്ന് അറിയുന്ന BERT മുതൽ OpenGPT3 വരെ ഉള്ള സ്റ്റേറ്റ് ഓഫ് ദി ആർട്ട് ട്രാൻസ്ഫോർമർ മോഡലുകളുടെ സൃഷ്ടിക്ക് വഴിതെളിച്ചു.

## ട്രാൻസ്ഫോർമർ മോഡലുകൾ

മുൻപ് ഓരോ മുൻ പ്രവചനത്തിന്റെ സാന്ദർഭം അടുത്ത മൂല്യനിർണയ ഘട്ടത്തിലേക്ക് മുന്നോട്ട് അയക്കുന്നതിന് പകരം, **ട്രാൻസ്ഫോർമർ മോഡലുകൾ** **സ്ഥാനം സൂചിപ്പിക്കുന്ന എൻകോഡിംഗുകളും** **ശ്രദ്ധയും** ഉപയോഗിച്ച് നൽകിയ ടെക്സ്റ്റ് വിൻഡോയിൽ ഉള്ള ഒരു ഇൻപുട്ടിന്റെ സാന്ദർഭം പിടിച്ചുപറയുന്നു. താഴെയുള്ള ചിത്രം, സ്ഥാനം സൂചിപ്പിക്കുന്ന എൻകോഡിംഗുകളും ശ്രദ്ധയും ഒരു നൽകിയ വിൻഡോയിൽ ഉള്ള സാന്ദർഭം എങ്ങനെ പിടിച്ചുപറയാമെന്ന് കാണിക്കുന്നു.

![Animated GIF showing how the evaluations are performed in transformer models.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

ഓരോ ഇൻപുട്ട് സ്ഥാനം സ്വതന്ത്രമായി ഓരോ ഔട്ട്പുട്ട് സ്ഥാനത്തേക്കും മാപ്പ് ചെയ്യപ്പെടുന്നതിനാൽ, ട്രാൻസ്ഫോർമറുകൾ RNN-കളേക്കാൾ മികച്ച പാരലലൈസേഷൻ സാധ്യമാക്കുന്നു, ഇത് വലിയതും കൂടുതൽ പ്രകടനശേഷിയുള്ള ഭാഷാ മോഡലുകൾ സൃഷ്ടിക്കാൻ സഹായിക്കുന്നു. ഓരോ ശ്രദ്ധാ തലയും വാക്കുകൾ തമ്മിലുള്ള വ്യത്യസ്ത ബന്ധങ്ങൾ പഠിക്കാൻ ഉപയോഗിക്കാം, ഇത് താഴെ വരുന്ന പ്രകൃതിഭാഷാ പ്രോസസ്സിംഗ് പ്രവർത്തനങ്ങൾ മെച്ചപ്പെടുത്തുന്നു.

## ലളിതമായ ട്രാൻസ്ഫോർമർ മോഡൽ നിർമ്മാണം

Keras-ൽ ട്രാൻസ്ഫോർമർ ലെയർ മുൻകൂട്ടി ഉൾപ്പെടുത്തിയിട്ടില്ല, പക്ഷേ നാം നമ്മുടെ സ്വന്തം മോഡൽ നിർമ്മിക്കാം. മുമ്പുപോലെ, AG News ഡാറ്റാസെറ്റിന്റെ ടെക്സ്റ്റ് വർഗ്ഗീകരണത്തിൽ ശ്രദ്ധ കേന്ദ്രീകരിക്കും, എന്നാൽ ട്രാൻസ്ഫോർമർ മോഡലുകൾ കൂടുതൽ പ്രയാസമുള്ള NLP പ്രവർത്തനങ്ങളിൽ മികച്ച ഫലം കാണിക്കുന്നുവെന്ന് പറയേണ്ടതാണ്.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Keras-യിലെ പുതിയ ലെയറുകൾ `Layer` ക്ലാസ്സിന്റെ സബ്ക്ലാസ്സായി ഉണ്ടാകണം, കൂടാതെ `call` മെത്തഡ് നടപ്പിലാക്കണം. **Positional Embedding** ലെയറുമായി തുടങ്ങാം. നാം [അധികൃത Keras ഡോക്യുമെന്റേഷനിൽ നിന്നുള്ള ചില കോഡ്](https://keras.io/examples/nlp/text_classification_with_transformer/) ഉപയോഗിക്കും. എല്ലാ ഇൻപുട്ട് സീക്വൻസുകളും `maxlen` നീളത്തിലേക്ക് പാഡ് ചെയ്തിട്ടുണ്ടെന്ന് നാം കരുതും.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

ഈ ലെയർ രണ്ട് `Embedding` ലെയറുകളടങ്ങിയതാണ്: ടോക്കണുകൾ (മുൻപ് ചർച്ച ചെയ്ത രീതിയിൽ) എമ്പെഡ് ചെയ്യുന്നതിനും ടോക്കൺ സ്ഥാനങ്ങൾ എമ്പെഡ് ചെയ്യുന്നതിനും. ടോക്കൺ സ്ഥാനങ്ങൾ 0 മുതൽ `maxlen` വരെ സ്വാഭാവിക സംഖ്യകളുടെ ഒരു ശ്രേണിയായി `tf.range` ഉപയോഗിച്ച് സൃഷ്ടിക്കപ്പെടുന്നു, പിന്നീട് എമ്പെഡിംഗ് ലെയറിലൂടെ കടന്നുപോകുന്നു. രണ്ട് ഫലമായ എമ്പെഡിംഗ് വെക്ടറുകളും ചേർത്ത്, `maxlen`$\times$`embed_dim` ആകൃതിയിലുള്ള സ്ഥാനാനുസൃതമായി എമ്പെഡുചെയ്ത ഇൻപുട്ട് പ്രതിനിധാനം സൃഷ്ടിക്കുന്നു.

<img src="../../../../../translated_images/ml/pos-embedding.e41ce9b6cf6078af.webp" width="40%"/>

ഇപ്പോൾ, ട്രാൻസ്ഫോർമർ ബ്ലോക്ക് നടപ്പിലാക്കാം. ഇത് മുൻപ് നിർവചിച്ച എമ്പെഡിംഗ് ലെയറിന്റെ ഔട്ട്പുട്ട് സ്വീകരിക്കും:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Transformer സ്ഥിതിവിവരക്കുറിപ്പുള്ള ഇൻപുട്ടിൽ `MultiHeadAttention` പ്രയോഗിച്ച് `maxlen`$\times$`embed_dim` വലുപ്പമുള്ള അറ്റൻഷൻ വെക്ടർ ഉത്പാദിപ്പിക്കുന്നു, അത് പിന്നീട് ഇൻപുട്ടുമായി മിശ്രിതമാക്കി `LayerNormalization` ഉപയോഗിച്ച് സാധാരണവത്കരിക്കുന്നു.

> **Note**: `LayerNormalization` *Computer Vision* ഭാഗത്ത് ചർച്ച ചെയ്ത `BatchNormalization` പോലെയാണ്, പക്ഷേ ഇത് ഓരോ ട്രെയിനിംഗ് സാമ്പിളിനും സ്വതന്ത്രമായി മുൻ ലെയറിന്റെ ഔട്ട്പുട്ടുകൾ സാധാരണവത്കരിച്ച് അവ [-1..1] പരിധിയിലേക്ക് കൊണ്ടുവരുന്നു.

ഈ ലെയറിന്റെ ഔട്ട്പുട്ട് പിന്നീട് `Dense` നെറ്റ്‌വർക്കിലൂടെ (നമ്മുടെ കേസിൽ - രണ്ട് ലെയർ പെർസെപ്ട്രോൺ) കടന്നുപോകുകയും, ഫലം അന്തിമ ഔട്ട്പുട്ടിൽ ചേർക്കുകയും ചെയ്യുന്നു (അത് വീണ്ടും സാധാരണവത്കരണത്തിലൂടെ കടന്നുപോകുന്നു).

<img src="../../../../../translated_images/ml/transformer-layer.905e14747ca4e7d5.webp" width="30%" />

ഇപ്പോൾ, നാം പൂർണ്ണമായ transformer മോഡൽ നിർവചിക്കാൻ തയ്യാറാണ്:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT ട്രാൻസ്ഫോർമർ മോഡലുകൾ

**BERT** (Bidirectional Encoder Representations from Transformers) ഒരു വളരെ വലിയ, 12 ലെയറുള്ള *BERT-base* മൾട്ടി ലെയർ ട്രാൻസ്ഫോർമർ നെറ്റ്‌വർക്കും, 24 ലെയറുള്ള *BERT-large* മോഡലും ആണ്. മോഡൽ ആദ്യം വലിയ വാചക ഡാറ്റാ കോർപ്പസിൽ (വിക്കിപീഡിയ + പുസ്തകങ്ങൾ) അനിയന്ത്രിത പരിശീലനത്തിലൂടെ (വാചകത്തിലെ മറച്ചുവച്ച വാക്കുകൾ പ്രവചിച്ച്) പ്രീ-ട്രെയിൻ ചെയ്യപ്പെടുന്നു. പ്രീ-ട്രെയിനിംഗിനിടെ മോഡൽ ഭാഷാ മനസ്സിലാക്കലിന്റെ ഒരു വലിയ തോതിൽ അറിവ് സമ്പാദിക്കുന്നു, പിന്നീട് ഇത് മറ്റ് ഡാറ്റാസെറ്റുകളുമായി ഫൈൻ ട്യൂണിങ്ങ് ഉപയോഗിച്ച് പ്രയോജനപ്പെടുത്താം. ഈ പ്രക്രിയയെ **ട്രാൻസ്ഫർ ലേണിംഗ്** എന്ന് വിളിക്കുന്നു.

![picture from http://jalammar.github.io/illustrated-bert/](../../../../../translated_images/ml/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

BERT, DistilBERT, BigBird, OpenGPT3 തുടങ്ങിയ നിരവധി ട്രാൻസ്ഫോർമർ ആർക്കിടെക്ചറുകളുടെ വ്യത്യാസങ്ങൾ ഉണ്ട്, അവ ഫൈൻ ട്യൂൺ ചെയ്യാവുന്നതാണ്.

നമ്മുടെ പരമ്പരാഗത സീക്വൻസ് ക്ലാസിഫിക്കേഷൻ പ്രശ്നം പരിഹരിക്കാൻ പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡൽ എങ്ങനെ ഉപയോഗിക്കാമെന്ന് നോക്കാം. [അധികൃത ഡോക്യുമെന്റേഷൻ](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) നിന്നുള്ള ആശയവും ചില കോഡും നാം കടത്തിക്കൊള്ളാം.

പ്രീ-ട്രെയിൻ ചെയ്ത മോഡലുകൾ ലോഡ് ചെയ്യാൻ, നാം **Tensorflow hub** ഉപയോഗിക്കും. ആദ്യം, BERT-നിഷ്ഠമായ വെക്ടറൈസർ ലോഡ് ചെയ്യാം:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

അസൽ നെറ്റ്‌വർക്കിന് പരിശീലിപ്പിച്ച വക്ടറൈസറിനൊപ്പം തന്നെ നിങ്ങൾ ഉപയോഗിക്കുന്നത് അത്യന്താപേക്ഷിതമാണ്. കൂടാതെ, BERT വക്ടറൈസർ മൂന്ന് ഘടകങ്ങൾ നൽകുന്നു:
* `input_word_ids`, ഇത് ഇൻപുട്ട് വാക്യത്തിനുള്ള ടോക്കൺ നമ്പറുകളുടെ ശ്രേണി ആണ്
* `input_mask`, ശ്രേണിയിലെ ഏത് ഭാഗം യഥാർത്ഥ ഇൻപുട്ട് ആണ്, ഏത് ഭാഗം പാഡിംഗ് ആണ് എന്ന് കാണിക്കുന്നു. ഇത് `Masking` ലെയർ സൃഷ്ടിക്കുന്ന മാസ്കിനോട് സമാനമാണ്
* `input_type_ids` ഭാഷാ മോഡലിംഗ് ടാസ്കുകൾക്കായി ഉപയോഗിക്കുന്നു, ഒരു ശ്രേണിയിൽ രണ്ട് ഇൻപുട്ട് വാക്യങ്ങൾ വ്യക്തമാക്കാൻ അനുവദിക്കുന്നു.

അപ്പോൾ, നാം BERT ഫീച്ചർ എക്സ്ട്രാക്ടർ ഇൻസ്റ്റാൻഷ്യേറ്റ് ചെയ്യാം:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


അപ്പോൾ, BERT ലെയർ പല ഉപകാരപ്രദമായ ഫലങ്ങൾ നൽകുന്നു:
* `pooled_output` എന്നത് സീക്വൻസിലെ എല്ലാ ടോക്കണുകളും ശരാശരി എടുത്ത ഫലമാണ്. ഇത് മുഴുവൻ നെറ്റ്വർക്കിന്റെ ബുദ്ധിമുട്ടുള്ള സെമാന്റിക് എംബെഡിങ്ങായി കാണാം. ഇത് നമ്മുടെ മുൻ മോഡലിലെ `GlobalAveragePooling1D` ലെയറിന്റെ ഔട്ട്പുട്ടിനോട് തുല്യമാണ്.
* `sequence_output` എന്നത് അവസാന ട്രാൻസ്ഫോർമർ ലെയറിന്റെ ഔട്ട്പുട്ടാണ് (മുകളിൽ കാണിച്ച മോഡലിലെ `TransformerBlock`-ന്റെ ഔട്ട്പുട്ടിനോട് തുല്യമാണ്)
* `encoder_outputs` എന്നത് എല്ലാ ട്രാൻസ്ഫോർമർ ലെയറുകളുടെയും ഔട്ട്പുട്ടുകളാണ്. നാം 4-ലെയർ BERT മോഡൽ ലോഡ് ചെയ്തിട്ടുണ്ട് (നാമത്തിൽ `4_H` ഉള്ളത് കാണുമ്പോൾ നിങ്ങൾക്ക് മനസ്സിലാകും), അതിനാൽ 4 ടെൻസറുകൾ ഉണ്ട്. അവസാനത്തേത് `sequence_output`-നോട് തുല്യമാണ്.

ഇപ്പോൾ നാം end-to-end ക്ലാസിഫിക്കേഷൻ മോഡൽ നിർവചിക്കും. മോഡൽ ഇൻപുട്ട് നിർവചിച്ച്, അതിന്റെ ഔട്ട്പുട്ട് കണക്കാക്കാൻ ഒരു പരമ്പര എക്സ്പ്രഷനുകൾ നൽകുന്ന *ഫംഗ്ഷണൽ മോഡൽ നിർവചന*ം ഉപയോഗിക്കും. BERT മോഡൽ വെയ്റ്റുകൾ ട്രെയിനബിൾ ആക്കാതെ, അവസാന ക്ലാസിഫയർ മാത്രം ട്രെയിൻ ചെയ്യുന്നതായി നമുക്ക് നിർവചിക്കാം:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


പരിശീലിക്കാവുന്ന പാരാമീറ്ററുകൾ കുറവായിട്ടും, പ്രക്രിയ വളരെ മന്ദഗതിയിലാണ്, കാരണം BERT ഫീച്ചർ എക്സ്ട്രാക്ടർ കണക്കുകൂട്ടൽ ഭാരം കൂടുതലാണ്. പരിശീലനത്തിന്റെ കുറവോ മോഡൽ പാരാമീറ്ററുകളുടെ കുറവോ കാരണം നാം യുക്തിസഹമായ കൃത്യത നേടാൻ കഴിഞ്ഞില്ലെന്ന് തോന്നുന്നു.

BERT വെയ്റ്റുകൾ അൺഫ്രീസ് ചെയ്ത് അതും പരിശീലിക്കാമെന്ന് നോക്കാം. ഇതിന് വളരെ ചെറിയ ലേണിംഗ് റേറ്റ് ആവശ്യമാണ്, കൂടാതെ **വാർംഅപ്പ്** ഉപയോഗിച്ച് കൂടുതൽ ശ്രദ്ധയോടെ പരിശീലന തന്ത്രം വേണം, **AdamW** ഓപ്റ്റിമൈസർ ഉപയോഗിച്ച്. ഓപ്റ്റിമൈസർ സൃഷ്ടിക്കാൻ `tf-models-official` പാക്കേജ് ഉപയോഗിക്കും:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

നിങ്ങൾക്ക് കാണാമല്ലോ, പരിശീലനം വളരെ മന്ദഗതിയിലാണ് നടക്കുന്നത് - പക്ഷേ നിങ്ങൾക്ക് പരീക്ഷണം നടത്താനും മോഡൽ കുറച്ച് എപ്പോക്കുകൾ (5-10) പരിശീലിപ്പിച്ച് മുമ്പ് ഉപയോഗിച്ച സമീപനങ്ങളുമായി താരതമ്യം ചെയ്ത് മികച്ച ഫലം നേടാമോ എന്ന് നോക്കാനും ആഗ്രഹിക്കാം.

## Huggingface Transformers ലൈബ്രറി

Transformer മോഡലുകൾ ഉപയോഗിക്കുന്ന മറ്റൊരു വളരെ സാധാരണവും (കുറച്ച് ലളിതവുമായ) മാർഗം [HuggingFace പാക്കേജ്](https://github.com/huggingface/) ആണ്, ഇത് വ്യത്യസ്ത NLP ടാസ്കുകൾക്കായി ലളിതമായ ബിൽഡിംഗ് ബ്ലോക്കുകൾ നൽകുന്നു. ഇത് Tensorflowക്കും PyTorchക്കും ലഭ്യമാണ്, മറ്റൊരു വളരെ പ്രശസ്തമായ ന്യൂറൽ നെറ്റ്‌വർക്ക് ഫ്രെയിംവർക്ക്.

> **Note**: നിങ്ങൾക്ക് Transformers ലൈബ്രറി എങ്ങനെ പ്രവർത്തിക്കുന്നു എന്ന് കാണാൻ താൽപ്പര്യമില്ലെങ്കിൽ - ഈ നോട്ട്‌ബുക്കിന്റെ അവസാനം വരെ പോകാം, കാരണം മുകളിൽ ചെയ്തതിൽ നിന്നു വലിയ വ്യത്യാസമില്ലാത്ത ഒന്നും നിങ്ങൾ കാണില്ല. വ്യത്യസ്തമായ ലൈബ്രറി ഉപയോഗിച്ച് BERT മോഡൽ പരിശീലിപ്പിക്കുന്ന സമാന ഘട്ടങ്ങൾ ഞങ്ങൾ ആവർത്തിക്കും, കൂടാതെ മോഡൽ വലുതാണ്. അതിനാൽ, പ്രക്രിയയിൽ കുറച്ച് ദൈർഘ്യമേറിയ പരിശീലനം ഉൾപ്പെടുന്നു, അതിനാൽ നിങ്ങൾക്ക് കോഡ് മാത്രം നോക്കിക്കാണാൻ ആഗ്രഹിക്കാം.

നമുക്ക് നോക്കാം നമ്മുടെ പ്രശ്നം [Huggingface Transformers](http://huggingface.co) ഉപയോഗിച്ച് എങ്ങനെ പരിഹരിക്കാമെന്ന്.


ആദ്യമായി ചെയ്യേണ്ടത് നാം ഉപയോഗിക്കാനിരിക്കുന്ന മോഡൽ തിരഞ്ഞെടുക്കുകയാണ്. ചില ബിൽറ്റ്-ഇൻ മോഡലുകളോടൊപ്പം, Huggingface-ൽ ഒരു [ഓൺലൈൻ മോഡൽ റിപോസിറ്ററി](https://huggingface.co/models) ഉണ്ട്, അവിടെ സമൂഹം ഒരുക്കിയ നിരവധി പ്രീ-ട്രെയിൻ ചെയ്ത മോഡലുകൾ കണ്ടെത്താം. ആ മോഡലുകൾ എല്ലാം മോഡൽ നാമം നൽകുന്നതിലൂടെ ലോഡ് ചെയ്ത് ഉപയോഗിക്കാം. മോഡലിനുള്ള എല്ലാ ആവശ്യമായ ബൈനറി ഫയലുകളും സ്വയം ഡൗൺലോഡ് ചെയ്യും.

ചിലപ്പോൾ നിങ്ങൾക്ക് നിങ്ങളുടെ സ്വന്തം മോഡലുകൾ ലോഡ് ചെയ്യേണ്ടി വരും, അപ്പോൾ ടോക്കനൈസറിന്റെ പാരാമീറ്ററുകൾ, മോഡൽ പാരാമീറ്ററുകളുള്ള `config.json` ഫയൽ, ബൈനറി വെയ്റ്റുകൾ എന്നിവ ഉൾപ്പെടെയുള്ള എല്ലാ ബന്ധപ്പെട്ട ഫയലുകളും അടങ്ങിയ ഡയറക്ടറി നിങ്ങൾ വ്യക്തമാക്കാം.

മോഡൽ നാമത്തിൽ നിന്ന്, നാം മോഡലും ടോക്കനൈസറും ഇരുവരും ഇൻസ്റ്റാൻഷ്യേറ്റ് ചെയ്യാം. ടോക്കനൈസറുമായി തുടങ്ങാം:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` ഒബ്ജക്റ്റിൽ `encode` ഫംഗ്ഷൻ ഉൾപ്പെടുന്നു, ഇത് ടെക്സ്റ്റ് എൻകോഡ് ചെയ്യാൻ നേരിട്ട് ഉപയോഗിക്കാം:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

നാം ഒരു സീക്വൻസിനെ മോഡലിലേക്ക് പാസ്സാക്കാൻ അനുയോജ്യമായ രീതിയിൽ എൻകോഡ് ചെയ്യാൻ ടോക്കനൈസർ ഉപയോഗിക്കാം, ഉദാ: `token_ids`, `input_mask` ഫീൽഡുകൾ ഉൾപ്പെടെ. നാം Tensorflow ടെൻസറുകൾ വേണമെന്ന് വ്യക്തമാക്കാൻ `return_tensors='tf'` ആർഗ്യുമെന്റ് നൽകാം:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

നമ്മുടെ കേസിൽ, നാം `bert-base-uncased` എന്ന മുൻകൂട്ടി പരിശീലിപ്പിച്ച BERT മോഡൽ ഉപയോഗിക്കും. *Uncased* എന്നത് മോഡൽ കേസ്-സെൻസിറ്റീവ് അല്ലെന്ന് സൂചിപ്പിക്കുന്നു.

മോഡൽ പരിശീലിപ്പിക്കുമ്പോൾ, നമുക്ക് ടോക്കൺ ചെയ്ത സീക്വൻസ് ഇൻപുട്ടായി നൽകേണ്ടതുണ്ട്, അതിനാൽ നാം ഡാറ്റ പ്രോസസ്സിംഗ് പൈപ്പ്‌ലൈൻ രൂപകൽപ്പന ചെയ്യും. `tokenizer.encode` ഒരു Python ഫംഗ്ഷൻ ആയതിനാൽ, കഴിഞ്ഞ യൂണിറ്റിൽ ചെയ്തതുപോലെ തന്നെ `py_function` ഉപയോഗിച്ച് അത് വിളിക്കും:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

ഇപ്പോൾ നാം `BertForSequenceClassfication` പാക്കേജ് ഉപയോഗിച്ച് യഥാർത്ഥ മോഡൽ ലോഡ് ചെയ്യാം. ഇത് നമ്മുടെ മോഡലിന് ക്ലാസിഫിക്കേഷനായി ആവശ്യമായ ആർക്കിടെക്ചർ, ഫൈനൽ ക്ലാസിഫയർ ഉൾപ്പെടെ, ഇതിനകം ഉണ്ടെന്ന് ഉറപ്പാക്കുന്നു. ഫൈനൽ ക്ലാസിഫയറിന്റെ വെയ്റ്റുകൾ ഇൻഷിയലൈസ് ചെയ്തിട്ടില്ലെന്നു കാണിക്കുന്ന മുന്നറിയിപ്പ് സന്ദേശം നിങ്ങൾക്ക് കാണാം, മോഡലിന് പ്രീ-ട്രെയിനിംഗ് ആവശ്യമുണ്ടാകും - അത് പൂർണ്ണമായും ശരിയാണ്, കാരണം അതാണ് നാം ചെയ്യാൻ പോകുന്നത്!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


`summary()`-ൽ നിന്നു കാണുന്നതുപോലെ, മോഡലിൽ ഏകദേശം 110 മില്യൺ പാരാമീറ്ററുകൾ ഉണ്ട്! സാധാരണയായി, ചെറിയ ഡാറ്റാസെറ്റിൽ ലളിതമായ ക്ലാസിഫിക്കേഷൻ ടാസ്ക് ചെയ്യാൻ ആഗ്രഹിക്കുന്നുവെങ്കിൽ, BERT ബേസ് ലെയർ ട്രെയിൻ ചെയ്യാൻ ആഗ്രഹിക്കില്ല:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


ഇപ്പോൾ നാം പരിശീലനം ആരംഭിക്കാൻ തയ്യാറാണ്!

> **Note**: പൂർണ്ണ തോതിലുള്ള BERT മോഡൽ പരിശീലനം വളരെ സമയം എടുക്കാം! അതിനാൽ നാം ആദ്യ 32 ബാച്ചുകൾക്ക് മാത്രമേ പരിശീലനം നടത്തൂ. ഇത് മോഡൽ പരിശീലനം എങ്ങനെ ക്രമീകരിക്കാമെന്ന് കാണിക്കാൻ മാത്രമാണ്. പൂർണ്ണ തോതിലുള്ള പരിശീലനം പരീക്ഷിക്കാൻ ആഗ്രഹിക്കുന്നുവെങ്കിൽ - `steps_per_epoch`യും `validation_steps`യും പാരാമീറ്ററുകൾ നീക്കം ചെയ്ത്, കാത്തിരിക്കാൻ തയ്യാറാകുക!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


നിങ്ങൾ ഇറ്ററേഷനുകളുടെ എണ്ണം വർദ്ധിപ്പിച്ച് മതിയായ സമയം കാത്തിരിക്കുകയും, പല എപ്പോക്കുകൾക്കായി പരിശീലനം നടത്തുകയും ചെയ്താൽ, BERT ക്ലാസിഫിക്കേഷൻ ഏറ്റവും മികച്ച കൃത്യത നൽകുമെന്ന് പ്രതീക്ഷിക്കാം! കാരണം BERT ഭാഷയുടെ ഘടന വളരെ നന്നായി മനസിലാക്കുന്നുണ്ട്, അതിനാൽ നാം അവസാന ക്ലാസിഫയർ മാത്രം ഫൈൻ-ട്യൂൺ ചെയ്യേണ്ടതുണ്ട്. എന്നാൽ, BERT ഒരു വലിയ മോഡലായതിനാൽ, മുഴുവൻ പരിശീലന പ്രക്രിയയും ഏറെ സമയം എടുക്കുകയും, ഗണനശേഷി (GPU, സാധാരണയായി ഒന്നിലധികം) ആവശ്യപ്പെടുകയും ചെയ്യുന്നു!

> **Note:** നമ്മുടെ ഉദാഹരണത്തിൽ, നാം ഏറ്റവും ചെറിയ പ്രീ-ട്രെയിൻ ചെയ്ത BERT മോഡലുകളിൽ ഒന്നാണ് ഉപയോഗിക്കുന്നത്. മികച്ച ഫലങ്ങൾ നൽകാൻ സാധ്യതയുള്ള വലിയ മോഡലുകളും ഉണ്ട്.


## പ്രധാനപ്പെട്ട കാര്യങ്ങൾ

ഈ യൂണിറ്റിൽ, നാം **ട്രാൻസ്ഫോർമറുകൾ** അടിസ്ഥാനമാക്കിയുള്ള ഏറ്റവും പുതിയ മോഡൽ ആർക്കിടെക്ചറുകൾ കണ്ടു. നാം അവയെ ടെക്സ്റ്റ് ക്ലാസിഫിക്കേഷൻ ടാസ്കിനായി ഉപയോഗിച്ചു, എന്നാൽ സമാനമായി BERT മോഡലുകൾ എന്റിറ്റി എക്സ്ട്രാക്ഷൻ, ചോദ്യോത്തരങ്ങൾ, മറ്റ് NLP ടാസ്കുകൾ എന്നിവയ്ക്കും ഉപയോഗിക്കാം.

ട്രാൻസ്ഫോർമർ മോഡലുകൾ NLP-യിലെ നിലവിലെ ഏറ്റവും മികച്ച സാങ്കേതികവിദ്യയെ പ്രതിനിധീകരിക്കുന്നു, കൂടാതെ അധികം കേസുകളിൽ കസ്റ്റം NLP പരിഹാരങ്ങൾ നടപ്പിലാക്കുമ്പോൾ പരീക്ഷണം ആരംഭിക്കേണ്ട ആദ്യ പരിഹാരമായിരിക്കണം. എന്നാൽ, ഈ മഡ്യൂളിൽ ചർച്ച ചെയ്ത റികറന്റ് ന്യൂറൽ നെറ്റ്‌വർക്കുകളുടെ അടിസ്ഥാന സിദ്ധാന്തങ്ങൾ മനസിലാക്കുന്നത്, നിങ്ങൾക്ക് ഉയർന്ന തലത്തിലുള്ള ന്യൂറൽ മോഡലുകൾ നിർമ്മിക്കാനാണ് ആഗ്രഹം എങ്കിൽ അത്യന്താപേക്ഷിതമാണ്.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**അസൂയാ**:  
ഈ രേഖ AI വിവർത്തന സേവനം [Co-op Translator](https://github.com/Azure/co-op-translator) ഉപയോഗിച്ച് വിവർത്തനം ചെയ്തതാണ്. നാം കൃത്യതയ്ക്ക് ശ്രമിച്ചിട്ടുണ്ടെങ്കിലും, യന്ത്രം ചെയ്ത വിവർത്തനങ്ങളിൽ പിശകുകൾ അല്ലെങ്കിൽ തെറ്റുകൾ ഉണ്ടാകാമെന്ന് ദയവായി ശ്രദ്ധിക്കുക. അതിന്റെ മാതൃഭാഷയിലുള്ള യഥാർത്ഥ രേഖ പ്രാമാണികമായ ഉറവിടമായി കണക്കാക്കണം. നിർണായക വിവരങ്ങൾക്ക്, പ്രൊഫഷണൽ മനുഷ്യ വിവർത്തനം ശുപാർശ ചെയ്യപ്പെടുന്നു. ഈ വിവർത്തനം ഉപയോഗിക്കുന്നതിൽ നിന്നുണ്ടാകുന്ന ഏതെങ്കിലും തെറ്റിദ്ധാരണകൾക്കോ വ്യാഖ്യാനക്കേടുകൾക്കോ ഞങ്ങൾ ഉത്തരവാദികളല്ല.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
